In [1]:
from utils.experiment import Experiment
exp = Experiment('point_mnist', 'swe', 'faiss-lsh', 9, random_state=0, num_slices=10)
acc = exp.test()
print(acc)
print(exp.get_time_report())

loading dataset...
dataset loaded!
preprocess samples...
compute referece...
loading cached base embedding...
compute query embedding


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:05<00:00, 1994.35it/s]


0.9243
